# House prices 

## Step 1: preparing data

In [1]:
import numpy as np
import pandas as pd 


In [16]:
train_og = pd.read_csv('data/train.csv')
test_og = pd.read_csv('data/test.csv')
x_train_raw = train_og.iloc[:,:-1]
y_train = train_og.iloc[:,-1]

In [28]:
def concat_df(train_data,test_data):
    return pd.concat([train_data,test_data]).reset_index(drop=True)

def divide_df(all_data):
    return all_data.iloc[:1460], all_data.iloc[1460:]

In [75]:
all_data = concat_df(x_train_raw,test_og)
all_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [43]:
taken_col = set(['ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond',
       'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinType2',
       'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating', 'HeatingQC',
       'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath',
       'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual', 'MasVnrType', 'MiscFeature', 'MiscVal', 'MoSold',
        'Neighborhood', 'OpenPorchSF', 'OverallCond', 'OverallQual',
       'PavedDrive', 'PoolArea', 'PoolQC', 'RoofMatl', 'RoofStyle',
        'SaleCondition', 'SalePrice', 'SaleType', 'ScreenPorch', 'Street',
        'TotRmsAbvGrd', 'TotalBsmtSF', 'Utilities', 'WoodDeckSF', 'YearBuilt',
        'YearRemodAdd', 'YrSold'])
all_col = set(all_data.columns)
my_col = list(all_col - taken_col)
len(my_col)

30

In [76]:
all_data[my_col].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2919 entries, 0 to 2918
Data columns (total 30 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Fireplaces     2919 non-null   int64  
 1   Exterior2nd    2918 non-null   object 
 2   FireplaceQu    1499 non-null   object 
 3   BldgType       2919 non-null   object 
 4   MasVnrArea     2896 non-null   float64
 5   GarageQual     2760 non-null   object 
 6   EnclosedPorch  2919 non-null   int64  
 7   LotFrontage    2433 non-null   float64
 8   GarageFinish   2760 non-null   object 
 9   LotShape       2919 non-null   object 
 10  Condition1     2919 non-null   object 
 11  GarageYrBlt    2760 non-null   float64
 12  LandContour    2919 non-null   object 
 13  GarageCond     2760 non-null   object 
 14  Fence          571 non-null    object 
 15  3SsnPorch      2919 non-null   int64  
 16  Exterior1st    2918 non-null   object 
 17  LotConfig      2919 non-null   object 
 18  Conditio

In [46]:
all_data[my_col].isnull().sum()

Fireplaces          0
Exterior2nd         1
FireplaceQu      1420
BldgType            0
MasVnrArea         23
GarageQual        159
EnclosedPorch       0
LotFrontage       486
GarageFinish      159
LotShape            0
Condition1          0
GarageYrBlt       159
LandContour         0
GarageCond        159
Fence            2348
3SsnPorch           0
Exterior1st         1
LotConfig           0
Condition2          0
LandSlope           0
GarageCars          1
Id                  0
LotArea             0
Alley            2721
GarageType        157
MSSubClass          0
GarageArea          1
MSZoning            4
Functional          2
HouseStyle          0
dtype: int64

**Exterior1st**

primary material 

In [91]:
all_data['Exterior1st'].value_counts()

Exterior1st
VinylSd    1025
MetalSd     450
HdBoard     442
Wd Sdng     411
Plywood     221
CemntBd     126
BrkFace      87
WdShing      56
AsbShng      44
Stucco       43
BrkComm       6
AsphShn       2
Stone         2
CBlock        2
ImStucc       1
Name: count, dtype: int64

In [113]:
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])

**Exterior2nd**

secondary cover material

In [77]:
all_data['Exterior2nd'].value_counts()

Exterior2nd
VinylSd    1014
MetalSd     447
HdBoard     406
Wd Sdng     391
Plywood     270
CmentBd     126
Wd Shng      81
BrkFace      47
Stucco       47
AsbShng      38
Brk Cmn      22
ImStucc      15
Stone         6
AsphShn       4
CBlock        3
Other         1
Name: count, dtype: int64

In [78]:
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna('Other')

**FireplaceQu**

NA if no fireplace

In [79]:
all_data['FireplaceQu'].value_counts()

FireplaceQu
Gd    744
TA    592
Fa     74
Po     46
Ex     43
Name: count, dtype: int64

In [80]:
all_data['FireplaceQu'] = all_data['FireplaceQu'].fillna('No')

**MasVnrArea**

only if MasVnrType is not nan 

In [81]:
all_data['MasVnrType'] = all_data['MasVnrType'].fillna(0)

In [82]:
all_data['MasVnrArea'].value_counts()

MasVnrArea
0.0      1738
120.0      15
176.0      13
200.0      13
216.0      12
         ... 
137.0       1
274.0       1
415.0       1
760.0       1
382.0       1
Name: count, Length: 444, dtype: int64

In [83]:
def masvnr_filler(masvnr): 
    if masvnr == 0: 
        return 0
    else: 
        return all_data['MasVnrArea'].mean()
    
all_data['MasVnrArea'][all_data['MasVnrArea'].isna()] = all_data['MasVnrType'][all_data['MasVnrArea'].isna()].apply(masvnr_filler)

/tmp/ipykernel_6751/3272868284.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data['MasVnrArea'][all_data['MasVnrArea'].isna()] = all_data['MasVnrType'][all_data['MasVnrArea'].isna()].apply(masvnr_filler)


In [85]:
all_data['MasVnrArea'].value_counts()

MasVnrArea
0.0      1761
120.0      15
176.0      13
200.0      13
216.0      12
         ... 
137.0       1
274.0       1
415.0       1
760.0       1
382.0       1
Name: count, Length: 444, dtype: int64

**Garage**

GarageType, GarageYrBlt, GarageFinish, GarageCars, GarageArea, GarageQual, GarageCond, GarageFinish

In [88]:
all_data['GarageArea'].describe()

count    2918.000000
mean      472.874572
std       215.394815
min         0.000000
25%       320.000000
50%       480.000000
75%       576.000000
max      1488.000000
Name: GarageArea, dtype: float64

In [94]:
all_data['GarageType']  = all_data['GarageType'].fillna('No')
all_data['GarageYrBlt']  = all_data['GarageYrBlt'].fillna(all_data['GarageYrBlt'].mean()) # есть явные выбросы!!! 
all_data['GarageYrBlt']  = all_data['GarageYrBlt'].fillna('No')
def garage_filler(gar): 
    if gar == 'No': 
        return 0
    else: 
        return all_data['GarageCars'].mode()
all_data['GarageCars'][all_data['GarageCars'].isna()] = all_data['GarageType'][all_data['GarageCars'].isna()].apply(garage_filler)
def garage_filler2(gar): 
    if gar == 'No': 
        return 0
    else: 
        return all_data['GarageArea'].mean()
all_data['GarageArea'][all_data['GarageArea'].isna()] = all_data['GarageType'][all_data['GarageArea'].isna()].apply(garage_filler2)
all_data['GarageQual']  = all_data['GarageQual'].fillna('No')
all_data['GarageCond']  = all_data['GarageCond'].fillna('No')
all_data['GarageFinish']  = all_data['GarageFinish'].fillna('No')

/tmp/ipykernel_6751/3938196724.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data['GarageCars'][all_data['GarageCars'].isna()] = all_data['GarageType'][all_data['GarageCars'].isna()].apply(garage_filler)
/tmp/ipykernel_6751/3938196724.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data['GarageArea'][all_data['GarageArea'].isna()] = all_data['GarageType'][all_data['GarageArea'].isna()].apply(garage_filler2)


**LotFrontage**

Linear feet of street connected to property

In [99]:
all_data['LotFrontage'].mean()

21.0

In [100]:
all_data['LotFrontage'] = all_data['LotFrontage'].fillna(all_data['LotFrontage'].mean())

**Fence**

na if no fence

In [116]:
all_data['Fence']  = all_data['Fence'].fillna('No')

**Alley**

na if no alley access

In [118]:
all_data['Alley']  = all_data['Alley'].fillna('No')

**MSZoning**


In [121]:
all_data['MSZoning'].value_counts()

MSZoning
RL         2265
RM          460
FV          139
RH           26
C (all)      25
Name: count, dtype: int64

In [122]:
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])

**Functional**

"assume typical"

In [123]:
all_data['Functional'].value_counts()

Functional
Typ     2717
Min2      70
Min1      65
Mod       35
Maj1      19
Maj2       9
Sev        2
Name: count, dtype: int64

In [124]:
all_data['Functional'] = all_data['Functional'].fillna(all_data['Functional'].mode()[0])

------------------------------

In [125]:
all_data[my_col].isnull().sum()

Fireplaces       0
Exterior2nd      0
FireplaceQu      0
BldgType         0
MasVnrArea       0
GarageQual       0
EnclosedPorch    0
LotFrontage      0
GarageFinish     0
LotShape         0
Condition1       0
GarageYrBlt      0
LandContour      0
GarageCond       0
Fence            0
3SsnPorch        0
Exterior1st      0
LotConfig        0
Condition2       0
LandSlope        0
GarageCars       0
Id               0
LotArea          0
Alley            0
GarageType       0
MSSubClass       0
GarageArea       0
MSZoning         0
Functional       0
HouseStyle       0
dtype: int64

In [127]:
all_data[my_col].to_csv('data/df_alex.csv')

## Step 2: joining parts and normalizing 

In [130]:
alex_df = pd.read_csv('data/df_alex.csv')
sveta_df = pd.read_csv('data/df_Sveta.csv')
roman_df = pd.read_csv('data/all_data_roman.csv')

In [184]:
joined_all.isnull().sum().sort_values(ascending=False)

MiscFeature     2814
MasVnrType      1766
SalePrice       1459
BsmtExposure      82
BsmtCond          82
                ... 
Alley              0
LotArea            0
Id                 0
GarageCars         0
KitchenQual        0
Length: 77, dtype: int64

In [192]:
joined_all = pd.concat([alex_df,sveta_df,roman_df],axis=1)


In [193]:
joined_all.drop('SalePrice',inplace=True, axis=1)

---------------

In [141]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, TargetEncoder, LabelEncoder

In [194]:
joined_all.BsmtFinType1 = joined_all.BsmtFinType1.fillna('None')
joined_all.BsmtFinType2 = joined_all.BsmtFinType2.fillna('None')
joined_all = joined_all.fillna('no')


In [195]:
joined_all.isna().sum()[joined_all.isna().sum()>0]

Series([], dtype: int64)

In [196]:
joined_all.dtypes

Unnamed: 0       int64
Fireplaces       int64
Exterior2nd     object
FireplaceQu     object
BldgType        object
                 ...  
FullBath         int64
HalfBath         int64
BedroomAbvGr     int64
KitchenAbvGr     int64
KitchenQual     object
Length: 76, dtype: object

In [197]:
cat_mask = (joined_all.nunique() < 10) | (joined_all.dtypes == object)
joined_all.loc[:,cat_mask]

,Fireplaces,Exterior2nd,FireplaceQu,BldgType,GarageQual,GarageFinish,LotShape,Condition1,LandContour,GarageCond,...,HeatingQC,CentralAir,Electrical,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual
0,0,VinylSd,No,1Fam,TA,RFn,Reg,Norm,Lvl,TA,...,Ex,Y,SBrkr,1,0,2,1,3,1,Gd
1,1,MetalSd,TA,1Fam,TA,RFn,Reg,Feedr,Lvl,TA,...,Ex,Y,SBrkr,0,1,2,0,3,1,TA
2,1,VinylSd,TA,1Fam,TA,RFn,IR1,Norm,Lvl,TA,...,Ex,Y,SBrkr,1,0,2,1,3,1,Gd
3,1,Wd Shng,Gd,1Fam,TA,Unf,IR1,Norm,Lvl,TA,...,Gd,Y,SBrkr,1,0,1,0,3,1,Gd
4,1,VinylSd,TA,1Fam,TA,RFn,IR1,Norm,Lvl,TA,...,Ex,Y,SBrkr,1,0,2,1,4,1,Gd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,0,CmentBd,No,Twnhs,No,No,Reg,Norm,Lvl,No,...,Gd,Y,SBrkr,0,0,1,1,3,1,TA
2915,0,CmentBd,No,TwnhsE,TA,Unf,Reg,Norm,Lvl,TA,...,TA,Y,SBrkr,0,0,1,1,3,1,TA
2916,1,VinylSd,TA,1Fam,TA,Unf,Reg,Norm,Lvl,TA,...,Ex,Y,SBrkr,1,0,1,0,4,1,TA
2917,0,Wd Shng,No,1Fam,No,No,Reg,Norm,Lvl,No,...,TA,Y,SBrkr,0,1,1,0,3,1,TA


In [198]:
le = LabelEncoder()

for col in joined_all.loc[:,cat_mask].columns:
    joined_all[col] = le.fit_transform(joined_all[col])
joined_all.loc[:,cat_mask]

,Fireplaces,Exterior2nd,FireplaceQu,BldgType,GarageQual,GarageFinish,LotShape,Condition1,LandContour,GarageCond,...,HeatingQC,CentralAir,Electrical,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual
0,0,13,3,0,5,2,3,2,3,5,...,0,1,4,1,0,2,1,3,1,2
1,1,8,5,0,5,2,3,1,3,5,...,0,1,4,0,1,2,0,3,1,3
2,1,13,5,0,5,2,0,2,3,5,...,0,1,4,1,0,2,1,3,1,2
3,1,15,2,0,5,3,0,2,3,5,...,2,1,4,1,0,1,0,3,1,2
4,1,13,5,0,5,2,0,2,3,5,...,0,1,4,1,0,2,1,4,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2914,0,5,3,3,3,1,3,2,3,3,...,2,1,4,0,0,1,1,3,1,3
2915,0,5,3,4,5,3,3,2,3,5,...,4,1,4,0,0,1,1,3,1,3
2916,1,13,5,0,5,3,3,2,3,5,...,0,1,4,1,0,1,0,4,1,3
2917,0,15,3,0,3,1,3,2,3,3,...,4,1,4,0,1,1,0,3,1,3


In [199]:
ss = StandardScaler()

joined_all = ss.fit_transform(joined_all)
pd.DataFrame(joined_all).columns

RangeIndex(start=0, stop=76, step=1)

In [201]:
joined_all = pd.DataFrame(joined_all)
joined_train, joined_test = divide_df(joined_all)
joined_train.to_csv('data/joined_train.csv')
joined_test.to_csv('data/joined_test.csv')

In [206]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

y_log = np.log(y_train)

lr = LinearRegression()

boost = GradientBoostingRegressor()
params = {
    'learning_rate': [0.01,0.05,0.1,0.2,0.4,0.8,1,2],
    'n_estimators': [5,10,15,20,30,40,50,80,100,200],
    'max_depth': [2,3,4,5,6]
}


cv = GridSearchCV(boost, 
                  param_grid=params, 
                  scoring='r2', 
                  cv=KFold(n_splits=5, random_state=1, shuffle=True), 
                  n_jobs=-1
                 )

cv.fit(joined_train, y_log)
cv.best_score_

0.896444561093347

In [219]:
res_example = pd.read_csv('data/sample_submission.csv')
res_example.index = x_test_raw['Id']
res_example

,Id,SalePrice
Id,,
1461,1461,169277.052498
1462,1462,187758.393989
1463,1463,183583.683570
1464,1464,179317.477511
1465,1465,150730.079977
...,...,...
2915,2915,167081.220949
2916,2916,164788.778231
2917,2917,219222.423400


In [213]:
cv.best_estimator_.predict(joined_test)


array([11.72960765, 11.90536892, 12.14266132, ..., 11.84767043,
       11.57555254, 12.34012787])

In [220]:
res_example['SalePrice'] = np.exp(cv.best_estimator_.predict(joined_test))
res_example.drop('Id', inplace=True, axis=1)


In [221]:
res_example

,SalePrice
Id,
1461,124194.933517
1462,148059.414728
1463,187711.469265
1464,187111.283549
1465,184610.590521
...,...
2915,71872.809264
2916,69520.026652
2917,139758.390395


In [222]:
res_example.to_csv('data/egor_team_first_try.csv')
